In [1]:
import pandas as pd
from datetime import datetime

import yfinance as yf

import plotly.graph_objects as go

In [2]:
# df = pd.read_csv("./data/EURUSD=X.csv", parse_dates=True)

df = yf.download("EURNZD=X", period="60d", interval="5m")

date = df.index

df['Date'] = date

print(df.head())

df['ema55'] = df['Close'].ewm(span=55).mean()
df['ema200'] = df['Close'].ewm(span=200).mean()

df['ema_diff'] = df['ema55'] - df['ema200']
df['crossover_type'] = df['ema_diff'].apply(
    lambda x: 'bullish' if x > 0 else 'bearish')

df = df.round(6)
df = df.sort_values(by='Date')
df = df.dropna()
df = df.reset_index(drop=True)


[*********************100%***********************]  1 of 1 completed
                              Open     High      Low    Close  Adj Close  \
Datetime                                                                   
2023-01-10 00:00:00+00:00  1.68502  1.68507  1.68356  1.68397    1.68397   
2023-01-10 00:05:00+00:00  1.68401  1.68470  1.68381  1.68429    1.68429   
2023-01-10 00:10:00+00:00  1.68433  1.68513  1.68417  1.68499    1.68499   
2023-01-10 00:15:00+00:00  1.68499  1.68553  1.68481  1.68522    1.68522   
2023-01-10 00:20:00+00:00  1.68522  1.68538  1.68378  1.68432    1.68432   

                           Volume                      Date  
Datetime                                                     
2023-01-10 00:00:00+00:00       0 2023-01-10 00:00:00+00:00  
2023-01-10 00:05:00+00:00       0 2023-01-10 00:05:00+00:00  
2023-01-10 00:10:00+00:00       0 2023-01-10 00:10:00+00:00  
2023-01-10 00:15:00+00:00       0 2023-01-10 00:15:00+00:00  
2023-01-10 00:20:00+00:00 

In [3]:
fig = go.Figure(
    data=[go.Candlestick(x=df['Date'],  open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']),
          go.Scatter(x=df['Date'], y=df['ema55'], name='ema55'),
          go.Scatter(x=df['Date'], y=df['ema200'], name='ema200')
          ])


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()


In [238]:
start_cash = 250000
trades = pd.DataFrame(columns=[
    'entry_date', 'exit_date', 'entry_price', 'exit_price', 'crossover_type', 'pnL', 'target', 'sl', 'balance'
])

position = {
    'entry_date': None,
    'exit_date': None,
    'entry_price': None,
    'exit_price': None,
    'is_long': False,
    'is_short': False,
    'active': False,
    'crossover_type': None,
    'target': None,
    'sl': None,
    'size': None,
    'balance': None,
    'pnL': None
}


def clear_position():
    position['entry_date'] = None
    position['exit_date'] = None
    position['entry_price'] = None
    position['exit_price'] = None
    position['is_long'] = False
    position['is_short'] = False
    position['active'] = False
    position['start_crossover_type'] = None
    position['target'] = None
    position['sl'] = None


def open(row, size=1, sl=0, target=0):
    if row is None:
        raise Exception("Provide opening trade data")

    position['entry_date'] = row['Date']
    position['entry_price'] = row['Close']
    position['is_long'] = row['crossover_type'] == 'bullish'
    position['is_short'] = row['crossover_type'] == 'bearish'
    position['active'] = True
    position['crossover_type'] = row['crossover_type']
    position['target'] = target
    position['sl'] = sl
    position['size'] = size
    position['balance'] = start_cash

    cost = position['entry_price'] * position['size']
    balance_amount = start_cash - cost

    if balance_amount < 0:
        raise Exception("Not enough balance to open position")

    position['cost'] = cost
    position['balance'] = balance_amount


def close(row, exit_price):
    if row is None:
        raise Exception("Provide closing trade data")

    if not position['active']:
        raise Exception("Cannot close an inactive position")

    position['exit_date'] = row['Date']
    # position['exit_price'] = row['Close']
    # position['pnL'] = (position['exit_price'] * position['size'] -
    #                    position['entry_price']) * position['size']

    position['exit_price'] = exit_price
    position['pnL'] = (exit_price - position['entry_price']) * position['size']

    if (position['is_short']):
        position['pnL'] = -position['pnL']

    position['balance'] = position['balance'] + position['pnL']

    global trades
    t_df = pd.DataFrame([position])
    trades = pd.concat([trades, t_df], ignore_index=True)
    clear_position()


df['position'] = None


In [239]:
start = 200
trades = pd.DataFrame(None)

while start < len(df):

    if position['active']:
        candle = df.iloc[start]

        # diff = candle['Close'] - position['entry_price']
        # print(pd.Series(position))

        candle_close = candle['Close']
        candle_entry = candle['Open']

        entry_price = position['entry_price']

        if position['is_long']:
            target_price = entry_price + position['target']
            stoploss = round(entry_price - position['sl'], 5)

            print("[Long]", entry_price, target_price, stoploss,
                  candle_close)

            if candle_close >= target_price:
                close(candle, target_price)

            if candle_close <= stoploss:
                close(candle, stoploss)

        if position['is_short']:
            target_price = entry_price - position['target']
            stoploss = round(entry_price + position['sl'], 5)

            print("[Short]", entry_price, target_price, stoploss,
                  candle_close)

            if (candle_close <= target_price):
                close(candle, target_price)

            if candle_close >= stoploss or candle_close <= entry_price:
                close(candle, stoploss)

           # Skip to next row if position is active
        start = start + 1
        continue

    if df['crossover_type'][start] != df['crossover_type'][start-1]:
        df['position'][start] = 'crossover'

        open(df.iloc[start], target=0.004, sl=0.001, size=100000)

    start = start + 1

print(start)


/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

[Short] 1.68807 1.68407 1.68907 1.68861
[Short] 1.68807 1.68407 1.68907 1.68943
[Long] 1.6893 1.6933 1.6883 1.69013
[Long] 1.6893 1.6933 1.6883 1.69014
[Long] 1.6893 1.6933 1.6883 1.68971
[Long] 1.6893 1.6933 1.6883 1.68842
[Long] 1.6893 1.6933 1.6883 1.68871
[Long] 1.6893 1.6933 1.6883 1.68847
[Long] 1.6893 1.6933 1.6883 1.68866
[Long] 1.6893 1.6933 1.6883 1.68865
[Long] 1.6893 1.6933 1.6883 1.68825
[Long] 1.69013 1.69413 1.68913 1.68969
[Long] 1.69013 1.69413 1.68913 1.6887
[Short] 1.68794 1.68394 1.68894 1.68843
[Short] 1.68794 1.68394 1.68894 1.68789
[Long] 1.69109 1.69509 1.69009 1.69068
[Long] 1.69109 1.69509 1.69009 1.69128
[Long] 1.69109 1.69509 1.69009 1.69073
[Long] 1.69109 1.69509 1.69009 1.69087
[Long] 1.69109 1.69509 1.69009 1.69041
[Long] 1.69109 1.69509 1.69009 1.69088
[Long] 1.69109 1.69509 1.69009 1.69062
[Long] 1.69109 1.69509 1.69009 1.69171
[Long] 1.69109 1.69509 1.69009 1.69107
[Long] 1.69109 1.69509 1.69009 1.69099
[Long] 1.69109 1.69509 1.69009 1.69098
[Long] 1.6

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26053/1190078983.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [242]:
profitable_trades = trades[trades['pnL'] > 0]
loss_trades = trades[trades['pnL'] < 0]
pnL = trades['pnL'].sum()
profit_factor = profitable_trades['pnL'].sum() / abs(loss_trades['pnL'].sum())

print("Profitable trades: ", len(profitable_trades),
      "Loss trades: ", len(loss_trades))
print("Profit factor: ", profit_factor)


Profitable trades:  13 Loss trades:  84
Profit factor:  0.6190476190476157
